In [ ]:
from datetime import datetime
import pandas as pd
import json
pd.set_option('display.max_columns', None)


In [ ]:
#2022-10-19T05:47:00
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')
df = pd.read_csv('../data/actual_data/raw/19.10.2022.csv', sep=';', parse_dates=['Arrival time', 'Arrival forecast', 'Departure time', 'Departure forecast'])
df = df.dropna(axis=0, subset=["Arrival time", "Arrival forecast"])

In [ ]:
df_sorted = df.sort_values(by=["Journey identifier", "Arrival time"])
df_sorted.head(10)

## Retards par ligne
* Pour chaque row
  + Si le journey identifier est le même que le précédent
    - Prendre la gare des deux row, et y associer le retard de la 2ème row
        

In [ ]:
df_merged = pd.concat([df_sorted, df_sorted.shift(-1).add_prefix('next_')], axis=1)

In [ ]:
class DelayForLine:
    station_a: str
    station_bb: str
    nb_train: int
    nb_delayed: int
    total_delay: int
    nb_cancelled: int

    def __init__(self, stations):
        self.station_a = stations[0]
        self.station_b = stations[1]
        self.nb_train = 0
        self.nb_delayed = 0
        self.total_delay = 0
        self.nb_cancelled = 0

    def add(self, is_delayed, delay, cancelled):
        self.nb_train += 1
        self.nb_delayed += is_delayed
        self.total_delay += delay # TODO maybe only if is_delayed is True
        self.nb_cancelled += cancelled

    def __repr__(self):
        return "DelayForLine: " + json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

In [ ]:

delay_by_line = {}

for i, row in df_merged.iterrows():
    if row['Journey identifier'] == row['next_Journey identifier']:
        is_delayed = row['Arrival delay']
        delay = abs(row['Arrival forecast'] - row['Arrival time']).total_seconds()
        depart_station = row['Stop name']
        arrival_station = row['next_Stop name']
        cancelled = row['Cancelled TF']
        sorted_stations = sorted([depart_station, arrival_station])
        line = '|'.join(sorted_stations)
        if line not in delay_by_line:
            delay_by_line[line] = DelayForLine(sorted_stations)
        delay_by_line[line].add(is_delayed, delay, cancelled)


In [ ]:
def obj_dict(obj):
    return obj.__dict__

with open("../data/actual_data/clean/delay_by_line.json", "w") as outfile:
    outfile.write(json.dumps(delay_by_line, default=obj_dict))

## Retards par gare
Pour chaque row
    Associer le retards à la gare

## Retard par jour de la semaine

## Retard par heure